In [ ]:
!wget https://dl.fbaipublicfiles.com/glue/superglue/data/v2/RTE.zip

--2023-01-30 09:44:08--  https://dl.fbaipublicfiles.com/glue/superglue/data/v2/RTE.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 750920 (733K) [application/zip]
Saving to: ‘RTE.zip.1’

RTE.zip.1           100%[===================>] 733.32K  1.75MB/s    in 0.4s    

2023-01-30 09:44:09 (1.75 MB/s) - ‘RTE.zip.1’ saved [750920/750920]



In [ ]:
!unzip RTE.zip

Archive:  RTE.zip
replace RTE/train.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [19]:
import pandas as pd

df_train = pd.read_json("RTE/train.jsonl", lines=True)

df_train.head()

,premise,hypothesis,label,idx
0,No Weapons of Mass Destruction Found in Iraq Yet.,Weapons of Mass Destruction Found in Iraq.,not_entailment,0
1,"A place of sorrow, after Pope John Paul II die...",Pope Benedict XVI is the new leader of the Rom...,entailment,1
2,Herceptin was already approved to treat the si...,Herceptin can be used to treat breast cancer.,entailment,2
3,"Judie Vivian, chief executive at ProMedica, a ...",The previous name of Ho Chi Minh City was Saigon.,entailment,3
4,A man is due in court later charged with the m...,Paul Stewart Hutchinson is accused of having s...,not_entailment,4


In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490 entries, 0 to 2489
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   premise     2490 non-null   object
 1   hypothesis  2490 non-null   object
 2   label       2490 non-null   object
 3   idx         2490 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 77.9+ KB


In [21]:
df_dev = pd.read_json("RTE/val.jsonl", lines=True)
df_dev.head()

,premise,hypothesis,label,idx
0,"Dana Reeve, the widow of the actor Christopher...",Christopher Reeve had an accident.,not_entailment,0
1,"Yet, we now are discovering that antibiotics a...",Bacteria is winning the war against antibiotics.,entailment,1
2,Cairo is now home to some 15 million people - ...,15 million tonnes of rubbish are produced dail...,not_entailment,2
3,"The Amish community in Pennsylvania, which num...",Pennsylvania has the biggest Amish community i...,not_entailment,3
4,Security forces were on high alert after an el...,Security forces were on high alert after a cam...,entailment,4


In [22]:
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   premise     277 non-null    object
 1   hypothesis  277 non-null    object
 2   label       277 non-null    object
 3   idx         277 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 8.8+ KB


In [23]:
df_test = pd.read_json("RTE/test.jsonl", lines=True)
df_test.head()

,premise,hypothesis,idx
0,Mangla was summoned after Madhumita's sister N...,Shukla is related to Mangla.,0
1,Authorities in Brazil say that more than 200 p...,Authorities in Brazil hold 200 people as hostage.,1
2,A mercenary group faithful to the warmongering...,An interior ministry worker was killed by a me...,2
3,"The British ambassador to Egypt, Derek Plumbly...",Derek Plumbly resides in Egypt.,3
4,Tibone estimated diamond production at four mi...,Botswana is a business partner of De Beers.,4


In [24]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   premise     3000 non-null   object
 1   hypothesis  3000 non-null   object
 2   idx         3000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 70.4+ KB


In [25]:
import numpy as np

df_test['label'] = np.nan

df_train['split'] = 'train'
df_dev['split'] = 'dev'
df_test['split'] = 'test'

df_all = pd.concat([df_train, df_dev, df_test])
df_all.head()

,premise,hypothesis,label,idx,split
0,No Weapons of Mass Destruction Found in Iraq Yet.,Weapons of Mass Destruction Found in Iraq.,not_entailment,0,train
1,"A place of sorrow, after Pope John Paul II die...",Pope Benedict XVI is the new leader of the Rom...,entailment,1,train
2,Herceptin was already approved to treat the si...,Herceptin can be used to treat breast cancer.,entailment,2,train
3,"Judie Vivian, chief executive at ProMedica, a ...",The previous name of Ho Chi Minh City was Saigon.,entailment,3,train
4,A man is due in court later charged with the m...,Paul Stewart Hutchinson is accused of having s...,not_entailment,4,train


In [28]:
def make_prompt(premise: str, hypothesis: str):
  return f'Having premise "{premise}" judge if the following hypothesis "{hypothesis}" are logically connected with the premise? \
          Answer "entailment" if yes, or "not_entailment" if no.'

In [29]:
prompts = []

for idx, row in df_all.iterrows():
  prompt = make_prompt(row['premise'], row['hypothesis'])
  prompts.append(prompt)


df_all['prompt'] = prompts
df_all['chatgpt_answer'] = np.nan

In [30]:
df_all.head()

,premise,hypothesis,label,idx,split,prompt,chatgpt_answer
0,No Weapons of Mass Destruction Found in Iraq Yet.,Weapons of Mass Destruction Found in Iraq.,not_entailment,0,train,"Having premise ""No Weapons of Mass Destruction...",NaN
1,"A place of sorrow, after Pope John Paul II die...",Pope Benedict XVI is the new leader of the Rom...,entailment,1,train,"Having premise ""A place of sorrow, after Pope ...",NaN
2,Herceptin was already approved to treat the si...,Herceptin can be used to treat breast cancer.,entailment,2,train,"Having premise ""Herceptin was already approved...",NaN
3,"Judie Vivian, chief executive at ProMedica, a ...",The previous name of Ho Chi Minh City was Saigon.,entailment,3,train,"Having premise ""Judie Vivian, chief executive ...",NaN
4,A man is due in court later charged with the m...,Paul Stewart Hutchinson is accused of having s...,not_entailment,4,train,"Having premise ""A man is due in court later ch...",NaN


In [31]:
df_all.to_csv("rte_prompts.tsv", sep="\t")

In [32]:
df_all_explain = df_all.copy()

df_all_explain['prompt'] = df_all_explain['prompt'].apply(lambda x: x + ' Explain your answer.')

df_all_explain.head()

,premise,hypothesis,label,idx,split,prompt,chatgpt_answer
0,No Weapons of Mass Destruction Found in Iraq Yet.,Weapons of Mass Destruction Found in Iraq.,not_entailment,0,train,"Having premise ""No Weapons of Mass Destruction...",NaN
1,"A place of sorrow, after Pope John Paul II die...",Pope Benedict XVI is the new leader of the Rom...,entailment,1,train,"Having premise ""A place of sorrow, after Pope ...",NaN
2,Herceptin was already approved to treat the si...,Herceptin can be used to treat breast cancer.,entailment,2,train,"Having premise ""Herceptin was already approved...",NaN
3,"Judie Vivian, chief executive at ProMedica, a ...",The previous name of Ho Chi Minh City was Saigon.,entailment,3,train,"Having premise ""Judie Vivian, chief executive ...",NaN
4,A man is due in court later charged with the m...,Paul Stewart Hutchinson is accused of having s...,not_entailment,4,train,"Having premise ""A man is due in court later ch...",NaN


In [33]:
df_all_explain.to_csv("rte_prompts_explain.tsv", sep="\t")

In [34]:
df_all_explain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5767 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   premise         5767 non-null   object 
 1   hypothesis      5767 non-null   object 
 2   label           2767 non-null   object 
 3   idx             5767 non-null   int64  
 4   split           5767 non-null   object 
 5   prompt          5767 non-null   object 
 6   chatgpt_answer  0 non-null      float64
dtypes: float64(1), int64(1), object(5)
memory usage: 360.4+ KB
